## 01 - notebook for simple feature finding based on opencv-python

#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 10th June 2025
#### edited - 
   

In [ ]:
from stemOrchestrator.logging_config   import setup_logging
data_folder  = "."
out_path = data_folder
setup_logging(out_path=out_path) 

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging
from typing import Dict
import os

In [ ]:
import os
import json
from pathlib import Path

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF_wd")
            port = port or secret.get("port_TF_wd")



config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,  # micro-seconds per pixel
    "haadf_resolution": 512, # square
    "out_path": "."
}

In [ ]:

# ip = config["ip"]
port = config["port"]
haadf_exposure = config["haadf_exposure"]
out_path = config["out_path"]
haadf_resolution = config["haadf_resolution"]



microscope = TemMicroscopeClient()
microscope.connect(ip, port = port)# 7521 on velox  computer

# query state:

tf_acquisition = TFacquisition(microscope=microscope)

# put beam shift to 0,0
# tf_acquisition.move_beam_shift_positon([0, 0])


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Acquire HAADF image and save TIFF
haadf_np_array, haadf_tiff_name = tf_acquisition.acquire_haadf(exposure=10e-6, resolution=512)

# Optional: retract HAADF detector
# tf_acquisition.haadf_det.retract()

# Convert TIFF to PNG (save for viewing)
HAADF_tiff_to_png(haadf_tiff_name)

# Normalize image for processing
img = haadf_np_array.astype(np.float32)
img = (img - img.min()) / (img.max() - img.min())  # Normalize to 0–1
img = (img * 255).astype(np.uint8)                 # Convert to 8-bit

# Apply Sobel edge detection
sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
edge_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
edge_magnitude = (edge_magnitude / edge_magnitude.max() * 255).astype(np.uint8)

# Display original and edge-detected images
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.title("Original HAADF")
plt.imshow(img, cmap='gray')
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Edge Map")
plt.imshow(edge_magnitude, cmap='gray')
plt.axis("off")

plt.tight_layout()
plt.show()
